# Search for HW Friendly Range

Goal is to find ranges/scales in a way so that from layer output to layer input, there is no need for multiplication & addition operation but rather a bit shifting multiplication. This gives some restrictions on the scales/ranges choice. To satisfy that here we only just the activation range. This opeartion is named weight focused, since the weights range is kept closer to the original range, so there is better accuracy for the weights.

### Import Packages

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models # type: ignore
from tensorflow.keras.applications import VGG16 # type: ignore
from tensorflow.keras.applications.vgg16 import preprocess_input # type: ignore
# Local imports
from ml_project_util.path import path_definition
from ml_project_util.flatten_model import flatten_condtitional
from ml_project_util.quantization_util import gen_sample_paths, wt_range_search, wt_scale_search
from ml_project_util.quantization_util import activation_range_search, activation_sw_scale_search, activation_hw_search
from ml_project_util.quantization_util import complete_dict_search

### Variable Paths, Execution Environments

In [ ]:
# dict = path_definition(ds_rel_path='catsdogsconv/CatsDogs')
dict = path_definition()
BASE_PATH = dict['BASE_PATH']
PATH_DATASET = dict['PATH_DATASET']
PATH_TEST = dict['PATH_TEST']
PATH_RAWDATA = dict['PATH_RAWDATA']
PATH_JOINEDDATA = dict['PATH_JOINEDDATA']
PATH_SAVEDMODELS = dict['PATH_SAVEDMODELS']

### Load Float Model

In [ ]:
# Two first letters for dataset, next letter stands for parameter
# 3 numbers are for the number of epochs, and the last 4 digits are for validation loss
# e.g. CD4_P2_FT_003_val0.0336
model_name = ''
short_name = model_name[:-10]
parent_name = model_name[:3]
filepath = f'{PATH_SAVEDMODELS}/{parent_name}/{model_name}.keras'
model = tf.keras.models.load_model(filepath)
model = flatten_condtitional(model, model_name)

### Find HW Activation Range for Conv and Dense Layers & Store it

In [ ]:
sampled_files = gen_sample_paths()
activation_sw_range_dict = activation_range_search(sampled_files, model, model_name, mode='s', force=0)

wt_range_dict = wt_range_search(model, model_name, mode='s')

wt_scale_dict = wt_scale_search(wt_range_dict, model_name, mode='s')

activation_sw_scale_dict = activation_sw_scale_search(activation_sw_range_dict, model_name, mode='s')

complete_dict = activation_hw_search(model_name, activation_sw_range_dict, activation_sw_scale_dict, wt_range_dict, wt_scale_dict, debug = 0, force=0)

### Function

In [ ]:
complete_dict = complete_dict_search(model, model_name, force=0, debug=0, mode='sv', filepath='0')